In [1]:
from datasets import load_dataset
from qdrant_client import models as qmodels
from qdrant_client import QdrantClient

ds = load_dataset("arbml/Hadith")


In [2]:
documents = ds['train']

## Load the model 

In [3]:
from pylate import indexes, models, retrieve

embedding_model = models.ColBERT(
    model_name_or_path="jinaai/jina-colbert-v2",trust_remote_code=True,device='cuda',
)

No sentence-transformers model found with name jinaai/jina-colbert-v2.
/home/linux/miniforge3/envs/kobo/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:984: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/linux/miniforge3/envs/kobo/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:1043: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
Loaded the ColBERT model from Stanford NLP.
The tokenizer does not support resizing the token embeddings, the prefixes token have not been added to vocabulary.


In [5]:
descriptions_embeddings = list(
    embedding_model.encode(documents['Text'],batch_size=64,show_progress_bar=True,device="cuda")
)

Encoding queries (bs=64):   0%|          | 0/1943 [00:00<?, ?it/s]

In [5]:
descriptions_embeddings[10].shape

(32, 128)

In [6]:
from qdrant_client import QdrantClient,models as qmodels

qdrant_client = QdrantClient(url="http://localhost:6333")

In [7]:
qdrant_client.create_collection(
    collection_name="try",
    vectors_config=qmodels.VectorParams(
        size=128,
        distance=qmodels.Distance.COSINE,
        multivector_config=qmodels.MultiVectorConfig(
            comparator=qmodels.MultiVectorComparator.MAX_SIM #similarity metric between multivectors (matrices)
        ),
    ),
    # quantization_config=qmodels.ScalarQuantization(
    #     scalar=qmodels.ScalarQuantizationConfig(
    #         type=qmodels.ScalarType.INT8,
    #         quantile=0.99,
    #         always_ram=True,
    #     ),
    # ),
)


True

In [11]:
documents[0]["Text"]

'أخبرنا قتيبة بن سعيد قال حدثنا سفيان عن الزهري عن أبي سلمة عن أبي هريرة أن النبي صلى الله عليه وسلم قال إذا استيقظ أحدكم من نومه فلا يغمس يده في وضوئه حتى يغسلها ثلاثا فإن أحدكم لا يدري أين باتت يده.'

In [14]:
qdrant_client.upload_points(
    collection_name="try",
    points = [
        qmodels.PointStruct(
            id=idx,
            vector=vector
        )
        for idx,vector in enumerate(descriptions_embeddings)
    ]
)

In [15]:
f_1 = "حديث جبريل الطويل عن الاسلام"
f_2 = "هل يجوز زيارة القبور؟"
f_3 = "أحاديث عن الصدقة"
f_4 = "هل يوجب المذي الغسل؟"

f_5 = "ماهي صفات المسلم؟"
h_1_t = "من ترك صلاة مكتوبة متعمدا برئت منه ذمة الله"  # 40165
h_2_t = "من سن في الإسلام سنة حسنة"  # 37492
h_3_t = "الراحمون يرحمهم الرحمن"  # 25311
h_4_t = "مثل المؤنين في توادهم وتراحمهم كمثل الجسد الواحد"  # 36753
h_5_t = "ألا إن في الجسد مضغة إذا صلحت صلح الجسد كله"  # 81731
h_6_t = "إن العبد ليتكلم بالكلمة من سخط الله"  # 27672
h_7_t = "من تشبه بقوم فهو منهم"  # 23973
h_8_t = "اجتنبوا أم الخبائث"  # 5573
h_9_t = "ليس المؤمن باللعان و لا الطعان ولا الفاحش"  # 22751

h_10_t = "فضل العالم علي العابد كفضل القمر"  # 39828
h_1 = "العهد الذي  بيننا وبينهم الصلاة فمن تركها فقد كفر"  # 78826
h_2 = "من دعا الي الهدي كان له من الأجور مثل أجور متبعه"  # 27900
h_3 = "من لا يرحم الناس لا يرحمه الله"  # 30040
h_4 = "المؤمن للمؤمن كالبنيان يشد بعضه بعضا"  # 37904
h_5 = "إن الله لا ينظر الي صوركم و لكن ينظرالي قلوبكم"  # 29642
h_6 = "من كان يؤمن بالله واليوم الأخر فليقل خيرا أو ليصمت"  # 25437
h_7 = "يحشر المرء مع من يحب"  #
h_8 = "لعن الله الخمر وشاربه وساقيها"  # 24563
h_9 = "سباب المسلم فسوق وقتاله كفر"  # 4037
h_10 = "من سلك طريقا يلتمس فيه علما"  # 27070

In [16]:
documents[0]

{'Text': 'أخبرنا قتيبة بن سعيد قال حدثنا سفيان عن الزهري عن أبي سلمة عن أبي هريرة أن النبي صلى الله عليه وسلم قال إذا استيقظ أحدكم من نومه فلا يغمس يده في وضوئه حتى يغسلها ثلاثا فإن أحدكم لا يدري أين باتت يده.'}

In [24]:
response = qdrant_client.query_points(
    collection_name="try",
    query=list(embedding_model.encode(h_2))[0], #converting generator object into numpy.ndarray
    limit=10, #How many closest to the query movies we would like to get
    #with_vectors=True, #If this option is used, vectors will also be returned
    with_payload=False#So metadata is provided in the output
)

for point in response.points:
    point_id = point.id
    score = point.score


    # Print or process each value
    # print(f"ID: {point_id}, Score: {score}")
    print(f"Hadith {documents[point_id]}")

Hadith {'Text': 'حدثنا عبد الوهاب الخفاف قال سئل سعيد عن الرجل يتبع جنازة ما له من الأجر فأخبرنا عن قتادة عن سالم بن أبي الجعد عن معدان بن أبي طلحة عن ثوبان مولى رسول الله صلى الله عليه وسلم أن النبي صلى الله عليه وسلم قال من صلى على جنازة فله قيراط فإن شهد دفنها فله قيراطان فسئل النبي صلى الله عليه وسلم عن ذلك القيراط فقال مثل أحد.'}
Hadith {'Text': 'و حدثني عن مالك عن ابن شهاب أنه قال من أهدى بدنة جزاء أو نذرا أو هدي تمتع فأصيبت في الطريق فعليه البدل.'}
Hadith {'Text': 'حدثنا أبو النعمان حدثنا أبو عوانة عن مغيرة عن إبراهيم قال في رجل أوصى بمثل نصيب بعض الورثة قال لا يجوز وإن كان أقل من الثلث قال أبو محمد هو حسن.'}
Hadith {'Text': 'و حدثني عن مالك عن يزيد بن خصيفة أنه سأل سليمان بن يسار عن رجل له مال وعليه دين مثله أعليه زكاة فقال لا.'}
Hadith {'Text': 'حدثنا عثمان بن أبي شيبة حدثنا جرير عن منصور عن أبي وائل قال قال عبد الله من حلف على يمين يستحق بها مالا لقي الله وهو عليه غضبان ثم أنزل الله تصديق ذلك { إن الذين يشترون بعهد الله وأيمانهم إلى عذاب أليم } ثم إن الأشعث بن قيس خرج إلينا ف

In [20]:
documents[36890]

{'Text': 'قال وكان يأتينا إذا قمنا إلى الصلاة فيمسح عواتقنا أو صدورنا وكان يقول لا تختلفوا فتختلف قلوبكم وكان يقول إن الله وملائكته يصلون على الصف الأول أو الصفوف الأول.'}

In [19]:
collection_info = qdrant_client.get_collection(collection_name="try")
print(f"Number of vectors; ",{collection_info.vectors_count})

Number of vectors;  {None}


memory_size = number_of_vectors * vector_dimension * 4 bytes * 1.5


In [30]:
x = len(documents)*128*4*1.5
x/1024/1024/1024

0.08893346786499023

In [31]:
memory_size = 1000000 * 1024 * 4  * 1.5 
memory_size/1024/1024/1024

5.7220458984375